# Pinhole Camera Model Practice Questions

## Question 1: Perspective Projection

A 3D point in the camera coordinate system is given by `p_c = [10, 5, 20]`. 
The camera has a focal length `f` of 50mm. 
Calculate the position of the projection of this point on the image plane in meters.

In [18]:
import numpy as np
def get_2d_point(p_c: list, f : float) -> list:
    u_c = (f * p_c[0])/p_c[2]
    v_c = (f * p_c[1])/p_c[2]
    return [u_c, v_c]
f = 50/1000
point2d = get_2d_point([10,5,20], 50)
point2d

[25.0, 12.5]

---

## Question 2: Conversion to Pixels

Given the projected point from Question 1, convert its coordinates from meters to pixels. 
Assume the following intrinsic parameters: 
- `s_x` = 800 pixels/meter
- `s_y` = 800 pixels/meter
- `o_x` = 320 pixels
- `o_y` = 240 pixels
What are the pixel coordinates `(u^I, v^I)`?

In [2]:
def convert_to_pixels(point2d: list, s_x, s_y, o_x, o_y) -> list:
    u_i = point2d[0]*s_x + o_x
    v_i = point2d[1]*s_y + o_y
    return [u_i,v_i]
    
s_x = 800
s_y = 800
o_x = 320
o_y = 240
pip2d = convert_to_pixels(point2d, s_x,s_y,o_x,o_y)
pip2d

[340.0, 250.0]

---

## Question 3: Intrinsic Matrix

Construct the intrinsic calibration matrix `K` using the parameters from Question 2, assuming no skew (`s_θ = 0`).

In [3]:
def get_intrinsic_matrix(f, s_x, s_y ,s_theta, o_x,o_y) -> list:
    return [
        [s_x*f,s_theta,o_x],
        [0, s_y*f, o_y],
        [0 , 0 , 1]]

k = get_intrinsic_matrix(f, s_x, s_y, 0, o_x, o_y)
k

[[40.0, 0, 320], [0, 40.0, 240], [0, 0, 1]]

In [3]:
def get_intrinsic_matrix(f, s_x, s_y ,s_theta, o_x,o_y) -> list:
    return [
        [s_x*f,s_theta,o_x],
        [0, s_y*f, o_y],
        [0 , 0 , 1]]

k = get_intrinsic_matrix(f, s_x, s_y, 0, o_x, o_y)
k

[[40.0, 0, 320], [0, 40.0, 240], [0, 0, 1]]

In [3]:
def get_intrinsic_matrix(f, s_x, s_y ,s_theta, o_x,o_y) -> list:
    return [
        [s_x*f,s_theta,o_x],
        [0, s_y*f, o_y],
        [0 , 0 , 1]]

k = get_intrinsic_matrix(f, s_x, s_y, 0, o_x, o_y)
k

[[40.0, 0, 320], [0, 40.0, 240], [0, 0, 1]]

---

## Question 4: Point Projection with Known Pose

A point in the world frame is given by `p_w = [2, 3, 50]`. The camera's pose `T_w^c` is given by an identity rotation matrix and a translation vector `t_w^c = [0, 0, 10]`. [cite_start]Using the intrinsic matrix `K` from Question 3, calculate the pixel coordinates of the projected point. [cite: 14, 16, 17]

In [19]:
def get_projection_matrix(i_mat, r_w, t_w):
    r_w = np.array(r_w)
    t_w = np.array(t_w).reshape(-1, 1) 
    
    e_mat = np.hstack([r_w,t_w])
    p_mat = np.matmul(i_mat,e_mat)
    return p_mat

def world_to_pixel(p_mat: list, coords: list):
    if len(coords) == 3:
        coords = coords + [1]
    coords = np.array(coords)
    
    mat = np.matmul(p_mat, coords)
    print(f"mat = {mat}") 
    pixel_x = mat[0] / mat[2]
    pixel_y = mat[1] / mat[2]
    return [pixel_x, pixel_y]

world_point = [2, 3, 50]
p_matrix = get_projection_matrix(k, [[1,0,0],[0,1,0],[0,0,1]], [0,0,10])
pixel_coords = world_to_pixel(p_matrix, world_point)
pixel_coords


mat = [19280. 14520.    60.]


[np.float64(321.3333333333333), np.float64(242.0)]

---

## Question 5: Radial Distortion

A point is projected on the image plane at `(u_distort^I, v_distort^I) = (400, 350)` including radial distortion. Given distortion coefficients `a1 = 0.0001` and `a2 = 0.0000002` and the principal point `(o_x, o_y) = (320, 240)`, calculate the corrected pixel coordinates `(u^I, v^I)`.

In [30]:
u_distort_I = 400
v_distort_I = 350
a1 = 0.0001
a2 = 0.0000002

r_2 = ((u_distort_I - o_x)**2 + (v_distort_I - o_y)**2)

scale = 1 + a1*r_2 + a2*(r_2**2)

u_c = scale * (u_distort_I - o_x) + o_x
v_c = scale * (v_distort_I - o_y) + o_y

(u_c,v_c)

(6024.0, 8083.0)